In [96]:
import json
import os
import pandas as pd

In [82]:
os.getcwd()

'/home/mike/nasahack/notebooks/anya'

In [3]:
mypath = "/home/mike/nasahack/notebooks/hubble/meta"

In [40]:
# get paths to all meta files
PATHS = []
for (dirpath, dirnames, filenames) in os.walk(mypath):
    PATHS.extend(filenames)
    break

In [41]:
PATHS = [x for x in PATHS if x != '1.json']

In [22]:
# get names of all images
names = {}

for path in PATHS:
    with open("/home/mike/nasahack/notebooks/hubble/meta/{path}".format(path=path)) as file:
        data = json.load(file)
        id_ = str(data['id'])
        names[id_] = data['name']
        file.close()

In [23]:
pics_names = list(names.values())

In [6]:
#!conda install nltk -y

In [24]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [94]:
# import nltk
# nltk.download('punkt')

In [25]:
pics_names_tok = []

for i in pics_names:
    toks = word_tokenize(i)
    for t in toks:
        t = ps.stem(t)
        pics_names_tok.append(t)

In [26]:
len(pics_names_tok)

29130

In [27]:
wordfreq = []
for w in pics_names_tok:
    wordfreq.append(pics_names_tok.count(w))

In [28]:
wfreqs = {}
for w,f in zip(pics_names_tok, wordfreq):
    wfreqs[w] = f

In [29]:
pd.DataFrame.from_dict(wfreqs, orient='index').sort_values(by=0,ascending=False).reset_index()['index'].tolist()[:100]

['of',
 'galaxi',
 'the',
 'imag',
 'hubbl',
 'and',
 'in',
 ')',
 '(',
 "'s",
 ',',
 'ngc',
 'star',
 'nebula',
 'cluster',
 '-',
 ':',
 'compass',
 'scale',
 'view',
 'for',
 'a',
 'from',
 'supernova',
 'A',
 'jupit',
 'artist',
 'comet',
 'field',
 'with',
 'spiral',
 'on',
 'hst',
 'light',
 'ring',
 'abel',
 'detail',
 '``',
 "''",
 'around',
 'disk',
 'mar',
 'univers',
 'dwarf',
 'deep',
 'space',
 'system',
 'saturn',
 'interact',
 'illustr',
 'to',
 'telescop',
 'moon',
 'globular',
 'region',
 'locat',
 'distant',
 'planet',
 'orion',
 'carina',
 'gravit',
 'infrar',
 'cloud',
 'at',
 'planetari',
 'ground-bas',
 'black',
 'annot',
 '2',
 'dust',
 'close-up',
 'stellar',
 'ga',
 'ski',
 '–',
 'milki',
 '1987a',
 'hole',
 'jet',
 'ac',
 'red',
 'way',
 'pluto',
 'color',
 'object',
 'collis',
 'core',
 'quasar',
 'survey',
 'composit',
 '1',
 'neptun',
 'len',
 "'",
 'impact',
 'HD',
 'center',
 'ultra',
 'SN',
 '9']

In [30]:
# labelling the images
# , "space":['comet', 'hole', 'space','moon','universe']
labels = {
    "galaxy":['galaxi', 'ring', 'quasar'],
    "star":['star', 'starburst','nebula', 'supernova','disk','orion','stellar'], 
    "planet":["planet", "jupyt","mar","saturn","neptun", "mercury", "venu", "earth", "uranu"]
         }

In [31]:
pics_names_tok = []

for i in pics_names:
    pics_name_tok = []
    toks = word_tokenize(i)
    for t in toks:
        t = ps.stem(t)
        pics_name_tok.append(t)
        
    pics_names_tok.append(pics_name_tok)

In [32]:
pics_names_tok[9]

['light', 'curv', 'of', 'cepheid', 'variabl', 'in', 'ngc', '3370']

In [33]:
labelled_pics = []

for t in pics_names_tok:
    if any(elem in t for elem in labels['galaxy']):
        labelled_pics.append("galaxy")
        continue
    elif any(elem in t for elem in labels['star']):
        labelled_pics.append("star")
        continue
    elif any(elem in t for elem in labels['planet']):
        labelled_pics.append("planet")
        continue
    else:
        labelled_pics.append("space")
        continue

In [34]:
len(labelled_pics)

4149

In [38]:
d = pd.DataFrame(labelled_pics, columns=['type'])
d.index = [x for x in list(names.keys()) if x != "1"]

4149

In [39]:
labels = pd.DataFrame.to_dict(d)['type']

In [136]:
with open("/home/mike/nasahack/notebooks/labels.json", "w") as outfile:
    json.dump(labels, outfile)
    outfile.close()

In [4]:
import json
with open("/home/mike/nasahack/notebooks/labels.json", "r") as outfile:
    data = json.load(outfile)
    outfile.close()

In [57]:
# get meta of all images
meta = {}

for path in PATHS:
    with open("/home/mike/nasahack/notebooks/hubble/meta/{path}".format(path=path)) as file:
        data = json.load(file)
        id_ = data['id']
        name = data['name']
        if ('description' not in list(data.keys())):
            meta[id_] = {'name':name}
        else:
            descr = data['description']
            meta[id_] = {'name':name, 'descr':descr}
        file.close()

In [55]:
'description' not in list(data.keys())

True

In [56]:
if ('description' not in list(data.keys())):
            print(1)

1


In [64]:
meta_df = pd.DataFrame.from_dict(meta, orient='index')


In [71]:
# labels_df = pd.DataFrame.from_dict(labels, orient='index')
with open("/home/mike/nasahack/notebooks/labels.json", "r") as outfile:
    data = json.load(outfile)
    labels_df = pd.DataFrame.from_dict(data,orient='index')
    outfile.close()

In [72]:
labels_df.columns = ['label']

In [88]:
labels_df.index = [int(x) for x in list(labels_df.index)]

In [93]:
meta_all = pd.merge(labels_df, meta_df, left_index=True, right_index=True, how='right').reset_index()
meta_all.columns = ['id','label','name','descr']

In [94]:
meta_all.to_csv('/home/mike/nasahack/notebooks/meta_all.csv', index=False)

In [114]:
gen_meta_all = pd.read_csv("/home/mike/nasahack/notebooks/misha/generated_meta.csv")
#gen_meta_all['name'] = "I'm sorry, you're wrong"
#gen_meta_all = 

In [107]:
import requests
res = requests.get("http://95.165.134.88:6007/gen.txt")
data = res.text
data = data.split()

In [108]:
len(data)

5168

In [125]:
gen_meta_all = gen_meta_all['descr'].tolist() * 4
gen_meta_all = gen_meta_all[:-464]

In [128]:
gen_meta_all = pd.DataFrame(gen_meta_all)
gen_meta_all['name'] = "I'm sorry, you're wrong"
gen_meta_all['id'] = data
gen_meta_all.columns = ["descr", 'name','id']

In [130]:
gen_meta_all.to_csv('/home/mike/nasahack/notebooks/gen_meta_all.csv', index=False)